Использование численных методов в задачах оптимизации

### 1. Поиск минимума функции:

\begin{align}
f(x)= \frac {1}{2} [ (x_ {1})^ {2} + \sum _ {i=1}^ {2} (x_ {i}-x_ {i+1})^ {2} + (x_ {3})^ {2} ]- x_ {1}
\end{align}


##### Представление применимости алгоритмов BFGS и L-BFGS к данной оптимизационной задаче

In [25]:
import numpy as np

In [4]:
def f(x, args_x_y):
    return 1/2 * (x[0] ** 2 + (x[0] - x[1]) ** 2 + (x[1] - x[2]) ** 2 + x[2] ** 2) - x[0]

Найдем градиент функции:
\begin{align} 2*x_1-1-x_2 \\  2*x_2-x_1-x_3\\  2*x_3-x_2\\ \end{align}

In [5]:
def grad_f(x, f, args_x_y):
    return np.array([-1 + x[0] + x[0] - x[1], -(x[0] - x[1]) + x[1] - x[2], x[2] - (x[1] - x[2])])

Найдем Гессиан:


\begin{array}{cc} 
 2 & -1 &  0\\
-1 &  2 & -1\\
 0 & -1 &  2\\
\end{array}


Найдем собственные значения Гессиана, если они будут положительны, то наша функция имеет хотя бы один локальный минимум.

In [27]:
Hess = np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
eig = np.linalg.eig(Hess)
print("Собственные значения матрицы Hessian:")
print(eig[0])

Собственные значения матрицы Hessian:
[3.41421356 2.         0.58578644]


#### Вывод

Данная функция без ограничений, дифференцируемая два раза, имеет хотя бы один локальный минимум, из этого следует, что она подходит для методов BFGS и L-BFGS.

##### Реализация алгоритмов BFGS и L-BFGS

In [4]:
def df(x, f, args_x_y):
    h = np.cbrt(np.finfo(float).eps)
    d = len(x)
    nabla = np.zeros(d)
    for i in range(d):
        x_for = np.copy(x)
        x_back = np.copy(x)
        x_for[i] += h
        x_back[i] -= h
        nabla[i] = (f(x_for, args_x_y) - f(x_back, args_x_y))/(2*h)
    return nabla

In [5]:
def pinpoint(alpha_low, alpha_high, x_0, p, df, f, args_x_y, mu_1 = 0.01, mu_2=0.09):
    k = 0
    while True:
        alpha_p = 0.5 * (alpha_high + alpha_low)
        if f(x_0 + alpha_p * p, args_x_y) > f(x_0, args_x_y) + mu_1 * alpha_p * df(x_0, f, args_x_y).T @ p or f(x_0 + alpha_p * p, args_x_y) > f(x_0 + alpha_low * p, args_x_y):
            alpha_high = alpha_p
        else:
            if np.abs(df(x_0 + alpha_p * p, f, args_x_y).T @ p) <= -mu_2*(df(x_0, f, args_x_y).T @ p):
                return alpha_p
            elif (df(x_0 + alpha_p * p, f, args_x_y).T @ p) * (alpha_high - alpha_low) >= 0:
                alpha_high = alpha_low
            alpha_low = alpha_p
        k += k


In [6]:
def linesearch(x_0, p, alpha, df, f, args_x_y, mu_1 = 0.01, mu_2=0.09, sigma=2.):
    alpha_1 = 0.
    alpha_2 = alpha
    first = True
    while True:
        if (f(x_0 + alpha_2 * p, args_x_y) > f(x_0, args_x_y) + mu_1 * alpha_2 * df(x_0, f, args_x_y).T @ p) or (not first and f(x_0 + alpha_2 * p, args_x_y) > f(x_0, args_x_y)):
            return pinpoint(alpha_1, alpha_2, x_0, p, df, f, args_x_y)
        if np.abs(df(x_0 + alpha_2 * p, f, args_x_y).T @ p) <= - mu_2 * df(x_0, f, args_x_y).T @ p:
            return alpha_2
        elif df(x_0 + alpha_2 * p, f, args_x_y).T @ p >= 0:
            return pinpoint(alpha_2, alpha_1, x_0, p, df, f, args_x_y)
        else:
            alpha_1, alpha_2 = alpha_2, sigma * alpha_2
        first = False

In [7]:
def BFGS(x_0, f, df, Hess, args_x_y=None, r = 1e-6):
    k = 0
    alpha = 1.
    N = len(x_0)

    while np.linalg.norm(df(x_0, f, args_x_y), np.inf) > r:
        if k == 0:
            V = Hess(df, f, x_0, args_x_y)
            x_1 = np.ones(N)
        else:
            s = x_1 - x_0
            y = df(x_1, f, args_x_y) - df(x_0, f, args_x_y)
            sigma = 1. / (s @ y)
            V = (np.eye(N) - sigma * np.outer(s, y)) @ V @ (np.eye(N) - sigma * np.outer(y, s)) + sigma * np.outer(s, s)
        p = - V @ df(x_0, f, args_x_y)
        alpha = linesearch(x_0, p, alpha, df, f, args_x_y)
        x_0, x_1 = x_1, x_0 + alpha * p

        k += 1
    return x_0, f(x_0, args_x_y), k

In [8]:
def compute_search_direction(df_x, s, y, sigma):
    alpha, beta = np.ones(len(s)), np.ones(len(s))
    d = df_x
    for i in range(len(s)-1, -1, -1):
        alpha[i] = sigma[i] * (s[i].T @ d)
        d = d - alpha[i] * y[i]

    V = (s[-1] @ y[-1])/(y[-1] @ y[-1])
    d = V * d

    for i in range(len(s)):
        beta[i] = sigma[i] * (y[i].T @ d)
        d = d + (alpha[i] - beta[i]) * s[i]
    return -d

In [9]:
def hess_eye(df, f, x_0, args_x_y):
    return 1. / np.linalg.norm(df(x_0, f, args_x_y), 1) * np.eye(len(x_0))


In [10]:
def L_BFGS(x_0, f, df,Hess, args_x_y=None, r = 1e-6):
    k = 0
    alpha = 1.
    N = len(x_0)
    s, y, sigma = [], [], []
    while np.linalg.norm(df(x_0, f, args_x_y), np.inf) > r:
        if k == 0:
            V = Hess(df, f, x_0, args_x_y)
            x_1 = np.ones(N)
            p = - V @ df(x_0, f, args_x_y)
            alpha = linesearch(x_0, p, alpha, df, f, args_x_y)
            x_0, x_1 = x_1, x_0 + alpha * p
        else:
            if len(s) > 6:
                s.pop(0)
                y.pop(0)
                sigma.pop(0)
            s.append(x_1 - x_0)
            y.append(df(x_1, f, args_x_y) - df(x_0, f, args_x_y))
            sigma.append(1. / (s[-1] @ y[-1]))
            df_x = df(x_0, f, args_x_y)
            p = compute_search_direction(df_x, s, y, sigma)
            alpha = linesearch(x_0, p, alpha, df, f, args_x_y)
            x_0, x_1 = x_1, x_0 + alpha * p

        k += 1


    return x_0, f(x_0, args_x_y), k

##### Приближение матрицы Гессе

In [11]:
def hessian_analitic(df, f, x, args_x_y):
    h = np.cbrt(np.finfo(float).eps)
    d = len(x)
    hessian = np.zeros((d, d))

    for i in range(d):
        xi = np.copy(x)
        xi[i] += h
        for j in range(d):
            x_for = np.copy(x)
            x_for[i] += h
            x_for[j] += h
            x_back = np.copy(x)
            x_back[j] += h
            hessian[i][j] = (f(x_for, args_x_y) - f(xi, args_x_y) - f(x_back, args_x_y) + f(x, args_x_y)) / (h ** 2)
    return hessian

##### Сравнение алгоритмов BFGS и L-BFGS

In [12]:
import time
def analize_time(metod):
    for i in [hess_eye, hessian_analitic]:
        print(i.__name__)
        t1 = time.time()
        x_0, f_x_0, k0 = metod(np.array([1.,1.,1.]), f, grad_f, i, r = 1e-6)
        t2 = time.time()
        print(f'Время выполнения {metod.__name__}, с градиентом, вычесленным аналитически: {t2-t1}', f'Минимум в точке: {x_0}', f'Значение минимума: {f_x_0}', f'Количество итераций: {k0}', sep='\n', end= '\n\n')

        t3 = time.time()
        x_1, f_x_1, k1 = metod(np.array([1.,1.,1.]), f, df, i, r = 1e-6)
        t4 = time.time()

        print(f'Время выполнения {metod.__name__}, с градиентом, вычесленным численно: {t4-t3}', f'Минимум в точке: {x_1}', f'Значение минимума: {f_x_1}', f'Количество итераций: {k1}', sep='\n', end= '\n\n')

In [13]:
analize_time(BFGS)

hess_eye
Время выполнения BFGS, с градиентом, вычесленным аналитически: 0.0072269439697265625
Минимум в точке: [0.74999987 0.50000008 0.25000016]
Значение минимума: -0.3749999999999544
Количество итераций: 10

Время выполнения BFGS, с градиентом, вычесленным численно: 0.006272077560424805
Минимум в точке: [0.74999987 0.50000008 0.25000016]
Значение минимума: -0.3749999999999545
Количество итераций: 10

hessian_analitic
Время выполнения BFGS, с градиентом, вычесленным аналитически: 0.0034265518188476562
Минимум в точке: [0.74999978 0.50000015 0.25000026]
Значение минимума: -0.37499999999986766
Количество итераций: 14

Время выполнения BFGS, с градиентом, вычесленным численно: 0.0066585540771484375
Минимум в точке: [0.74999978 0.50000015 0.25000026]
Значение минимума: -0.3749999999998676
Количество итераций: 14



In [14]:
analize_time(L_BFGS)

hess_eye
Время выполнения L_BFGS, с градиентом, вычесленным аналитически: 0.002423524856567383
Минимум в точке: [0.75000031 0.50000006 0.25000003]
Значение минимума: -0.3749999999999225
Количество итераций: 10

Время выполнения L_BFGS, с градиентом, вычесленным численно: 0.01538705825805664
Минимум в точке: [0.75000031 0.50000006 0.25000003]
Значение минимума: -0.3749999999999225
Количество итераций: 10

hessian_analitic
Время выполнения L_BFGS, с градиентом, вычесленным аналитически: 0.002747058868408203
Минимум в точке: [0.75000061 0.50000027 0.25000017]
Значение минимума: -0.37499999999973993
Количество итераций: 10

Время выполнения L_BFGS, с градиентом, вычесленным численно: 0.00700831413269043
Минимум в точке: [0.75000061 0.50000027 0.25000017]
Значение минимума: -0.37499999999974
Количество итераций: 10



По времени работы и итерациям видно, что метод L-BFGS быстрее справляется с задачей мнимизации данной функции, при чем когда градиент вычеслен аналитически. Также не сложно догадаться, что он не затрачивает память на хранение матрицы Гессе.

Самое наименьшее время работы BFGS было при задании Гесссиана методом конечных разностей.

Так вычислительная сложность BFGS составляет O(n^2), в то время как LBFGS - O(n)

### 2. Практическое применение: использование метода оптимизации L-BFGS для решения задачи логистической регрессии

##### Представление набора данных

In [6]:
from sklearn.model_selection import cross_val_score, train_test_split
import os
import pandas as pd

Данный датасет содержит цены и другие атрибуты почти по 54 000 бриллиантам.

**Описание переменных:

price -- цена в US dollars

carat -- вес бриллианта

cut -- качество среза (Fair, Good, Very Good, Premium, Ideal)

color diamond -- цвет, от J (худший) до D (лучший)

clarity -- мера чистоты бриллианта (I1 (худший), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (лучший))

x — длина в мм

y — ширина в мм

z — глубина в мм

depth — общая глубина в процентах

table — ширина вершины бриллианта относительно самой широкой точки

In [7]:
data = pd.read_csv(os.path.join('diamonds_moded.xls'), sep=';')
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,NaN,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [8]:
from sklearn import preprocessing

In [9]:
data['color'].fillna(data['color'].mode()[0], inplace=True)
cat_columns = [cname for cname in data.columns if data[cname].dtype == 'object']
encoder = preprocessing.LabelEncoder()
for col in cat_columns:
    data[col] = encoder.fit_transform(data[col])
X = data.drop('price', axis=1)
y = data['price']

In [10]:
y = pd.Series(1 if i > 1000 else 0 for i in y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

##### Реализация логистической регрессии

In [24]:
class LogReg():

    def __init__(self):
        self.thetas = None
        self.loss_history = []

    def fit_BFGS(self, x, y):
        x, y = x.copy(), y.copy()
        self.add_ones(x)
        thetas, n = np.zeros(x.shape[1]), x.shape[0]
        self.thetas = BFGS( thetas, self.objective, df=self.gradient, Hess=hess_eye, args_x_y=(x, y))[0]

    def fit_L_BFGS(self, x, y):
        x, y = x.copy(), y.copy()
        self.add_ones(x)
        thetas, n = np.zeros(x.shape[1]), x.shape[0]
        self.thetas = L_BFGS( thetas, self.objective, df=self.gradient, Hess=hess_eye, args_x_y=(x, y))[0]

    def predict(self, x):
        x = x.copy()
        self.add_ones(x)
        z = np.dot(x, self.thetas)
        probs = np.array([self.stable_sigmoid(value) for value in z])
        return np.where(probs >= 0.5, 1, 0)

    def add_ones(self, x):
        x.insert(0, 'x0', np.ones(x.shape[0]))

    def h(self, thetas, x):
        z = np.dot(x, thetas)
        return np.array([self.stable_sigmoid(value) for value in z])

    def objective(self, thetas, args):
        (x, y) = args
        y_pred = self.h(thetas, x)
        y_one_loss = y * np.log(y_pred + 1e-9)
        y_zero_loss = (1 - y) * np.log(1 - y_pred + 1e-9)
        return -np.mean(y_zero_loss + y_one_loss)

    def gradient(self, thetas, f, args):
        (x, y) = args
        y_pred = self.h(thetas, x)
        return np.dot(x.T, (y_pred - y)) / x.shape[0]

    def stable_sigmoid(self, z):
        if z >= 0:
            return 1 / (1 + np.exp(-z))
        else:
            return np.exp(z) / (np.exp(z) + 1)


##### Сравнение характеристики работы алгоритмов

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
model = LogReg()
t1 = time.time()
model.fit_BFGS(X_train, y_train)
t2 = time.time()
print(t2-t1)
y_pred_BFGS = model.predict(X_valid)


0.2463207244873047


In [31]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_valid, y_pred_BFGS)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7048419190802565


In [29]:
model = LogReg()
t1 = time.time()
model.fit_L_BFGS(X_train, y_train)
t2 = time.time()
print(t2-t1)
y_pred_L_BFGS = model.predict(X_valid)

0.3538699150085449


In [33]:
accuracy = accuracy_score(y_valid, y_pred_L_BFGS)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7048419190802565


Оба метода оптимизации показывают хорошие результаты и предсказывают более 70% данных, но при этом логистическая регрессия с методом BFGS обучается быстрее.

Сравнение с Логистической регрессией из sklearn

In [11]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression().fit(X_train, y_train)

print(f'Accuracy:{accuracy_score(y_valid, model.predict(X_valid))}')

Accuracy:0.9526325546903968
